<a href="https://colab.research.google.com/github/FedeEzeq/ML2---Chatbot-en-Google-Cloud/blob/main/Lambda_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
import unicodedata
import spacy_stanza
import requests
import numpy as np
import re

os.system("gsutil cp gs://chatbot_ml2/words.pkl words.pkl")
os.system("gsutil cp gs://chatbot_ml2/classes.pkl classes.pkl")

words_file = open("words.pkl",'rb')
words = pickle.load(words_file)
words_file.close()

classes_file = open("classes.pkl",'rb')
classes = pickle.load(classes_file)
classes_file.close()

nlp = spacy_stanza.load_pipeline("es")

def preprocess_clean_text(text):    
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(pattern, '', text)
    return text
  
def text_to_tokens(text): 
    lemma_tokens = []
    tokens = nlp(preprocess_clean_text(text.lower()))
    for token in tokens:
        lemma_tokens.append(token.lemma_)
    return lemma_tokens

def bag_of_words(text, vocab): 
    tokens = text_to_tokens(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)


def mensaje_alumno (external_request):
  
  headers = {}
  external_request = external_request.json
  data = external_request['data']
  data2 = bag_of_words(data,words)
 
 
  #columns = external_request['columns']
  headers = {
    'Content-Type': 'application/json',
        }

  json_data = {"columns": ["X"],"data": data2}
  response = requests.post('http://34.134.63.173:5000/invocations', headers=headers, json=json_data)

  thresh = 0.2
        y_pred = [[idx, res] for idx, res in enumerate(response) if res > thresh] 
        y_pred.sort(key=lambda x: x[1], reverse=True)
    
        for r in y_pred:
            return_list.append(labels[r[0]])




  
  return str(response.text)

#----------------------------------------------------------------------------------------------------------

#MODEL PREDICT 
def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    words_recognized = sum(bow)

    return_list = []
    if words_recognized > 0:
        result = model.predict(np.array([bow]))[0] #vector de largo = cantidad de clases, 
        thresh = 0.2
        y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh] 
        y_pred.sort(key=lambda x: x[1], reverse=True)
    
        for r in y_pred:
            return_list.append(labels[r[0]])
            #print(labels[r[0]], r[1])

    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = "BOT: " + random.choice(i["responses"])
            break
    return result


#INTERACCION CON ALUMNO
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    if len(intents) > 0:
        result = get_response(intents, dataset)
        print(result)
    else:
        print("Perdón, no comprendo la pregunta.")
    